In [50]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact


In [21]:
working_dir = '/home/Work/clusters/'
all_craters = pd.read_csv(os.path.join(working_dir,'all_data_clusters.csv'))

f = os.path.join(working_dir, 'ndarray_of_all_craters.txt')

files = np.loadtxt(os.path.join(working_dir, 'filename_list.txt'), dtype='str')
total_f = len(files)
total_c = 3


data_all = np.loadtxt(f).reshape((total_f, total_c, 1100))
print(data_all.shape)

(709, 3, 1100)


In [22]:
k=1
bins=[k]
for i in range(11):
    k*= 2**(0.5)
    bins.append(k)

mid_bin = []
bin_width = []
for i in range(len(bins)-1):
    mid_bin.append( (bins[i] + (bins[i+1]-bins[i]) / 2  ) )
    bin_width.append(bins[i+1]-bins[i])

bin_width = np.array(bin_width)
mid_bin = np.array(mid_bin)


In [ ]:
plt.close()


def interact_plot(N, upper_limit, lower_limit, remove_start):#, remove_end):
    plt.figure(figsize=(8,6))
    dsfd_values=np.empty([len(bins)-1, total_f])*np.nan
    count=0
    color=iter(plt.cm.gist_ncar(np.linspace(0,1,N)))

    for i in range(total_f):
        
        p_data = np.array([n for n in data_all[i, :,:][0] if n>0.005])
        if len(p_data)>lower_limit and count<N and len(p_data)<upper_limit:
            c=next(color)
            
            try:
                area = all_craters.loc[all_craters["Observation_ID"]== files[i], "area_ellipse"].iloc[0]
                area /=1e6
            except IndexError:
                print(f'the crater {files[i]} was not found in the all_craters csv file. using an area of 1 km2')
                area = 1/1e6
            n = np.array([np.sum((p_data>bins[i])&((p_data<=bins[i+1]))) for i in range(len(bins)-1)])
            
            d_vals = n/bin_width

            plt.plot( mid_bin, d_vals, '.', label=files[i], c=c)
            
            # plot error bars
            stds = n**0.5
            min_n = n - stds
            max_n = n + stds
            
            plt.vlines(mid_bin,min_n/bin_width,max_n/bin_width, color=c, alpha = 0.1)


            #line fitting
            reduced_n_mask = (n>3)       # take only bins with craters over this value
            for r in range(remove_start):
                reduced_n_mask[r] = False    # removing first bin points as given
#             k = 0
#             r = 0
#             while r<remove_end: 
#                 print(r, k, reduced_n_mask[-k])
#                 if reduced_n_mask[-k] == True:    # removing end points where not already false
#                     reduced_n_mask[-k] = False
#                     r+=1
#                 k+=1
#             print(reduced_n_mask)

            log_x_data = np.log10(mid_bin[reduced_n_mask])
            log_y_data = np.log10(d_vals[reduced_n_mask])

            a, b = np.polyfit(log_x_data, log_y_data, 1)
            y = 10**(a * np.log10(mid_bin) + b)


            plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}', c=c)
            count+=1



    plt.xscale("log")
    plt.yscale("log")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid()

print("To use this interact tool:\nDetermin the number of clusters you wish to plot (N) \nthat contain a subset of craters within the upper and lower limit values.\nThe remove from start option allow you to adjust the linear fits.\nNote that any bins with less than 3 craters are already removed from the fit")
interact(interact_plot, N=[1, 5, 10, 1000], upper_limit=(5, 600, 10), lower_limit=(5, 400, 10), remove_start=[1, 0, 2, 3, 4, 5])#, remove_end=[1, 0, 2, 3, 4, 5])

# same again, but without interact

In [ ]:
plt.close()
plt.figure(figsize=(8,6))
dsfd_values=np.empty([len(bins)-1, total_f])*np.nan

limit = 50
count=0

for i in range(total_f):
    p_data = np.array([n for n in data_all[i, :,:][0] if n>0.005])
    if len(p_data)>limit:
        try:
            area = all_craters.loc[all_craters["Observation_ID"]== files[i], "area_ellipse"].iloc[0]
            area /=1e6
        except IndexError:
            area = 1/1e6
        n = np.array([np.sum((p_data>bins[i])&((p_data<=bins[i+1]))) for i in range(len(bins)-1)])
        stds = n**0.5
        min_n = n - stds
        max_n = n + stds
        #n,x,_ = plt.hist(p_data, bins = bins, histtype='step', color='k', alpha=0)#, normed=1)
        d_vals = n/bin_width
#         mid_x = []
#         for i in range(len(x)-1):TypeError: 'str' object isTypeError: 'str' object is not callable not callable
#             #mid_val = (x[i] + (x[i+1]-x[i]) / 2  ) 
#             mid_x.append( mid_val )
#             d_vals.append(area / mid_val)

        plt.plot( mid_bin, d_vals, '.', label=files[i])
#         plt.vlines(mid_bin,min_n/bin_width,max_n/bin_width)


        #line fitting
        reduced_n_mask = (n>2)       # take only bins with craters over this value
        reduced_n_mask[0] = False    # removing first bin points where <1m is unreliable

        log_x_data = np.log10(mid_bin[reduced_n_mask])
        log_y_data = np.log10(d_vals[reduced_n_mask])

        a, b = np.polyfit(log_x_data, log_y_data, 1)
        y = 10**(a * np.log10(mid_bin) + b)
#         y = 10B * 10**(a* mid_bin[reduced_n_mask])

#         log10(N) = -a log10(m) + log10(b)
#          N = 10** (-a log10(m) + log10(b))
#         y= 10**b * 10**(a*xdata)

        plt.plot(mid_bin, y, label=f'slope of fit line:{a:.3f}, N craters:{len(p_data)}')
        count+=1



plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid()
